<a href="https://colab.research.google.com/github/pamelamagotte-max/PamelaSiteRAG/blob/main/tools/crawl_website.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
# 1) Installer les libs
!pip -q install requests beautifulsoup4 markdownify trafilatura python-slugify tldextract tqdm

# 2) Le script du crawler (copié tel quel ici)
START_URL = "https://pamelamagotte.fr/"   # <- change si besoin
MAX_PAGES = 400                            # <- mets la limite que tu veux

import os, re, time, hashlib, requests
from urllib.parse import urljoin, urlparse
from bs4 import BeautifulSoup
from markdownify import markdownify as md
from slugify import slugify
from tqdm import tqdm

OUT_DIR = "corpus"

def same_host(url, base): return urlparse(url).netloc == urlparse(base).netloc
BAD_EXT = (".pdf",".jpg",".jpeg",".png",".gif",".webp",".svg",".zip",".mp3",".mp4",".avi",".mov",".wmv",".ico")
HEADERS = {"User-Agent": "PamelaBot/1.0 (+https://assistant-pamela.streamlit.app)"}

def fetch(u):
    try:
        r = requests.get(u, headers=HEADERS, timeout=20)
        if r.status_code == 200 and "text/html" in r.headers.get("Content-Type",""):
            return r.text
    except: pass
    return None

def clean_html(html, base_url):
    soup = BeautifulSoup(html, "html.parser")
    for tag in soup(["script","style","noscript","iframe"]): tag.decompose()
    for tag in soup.select("nav, footer, aside, form"): tag.decompose()
    title = soup.title.string.strip() if soup.title and soup.title.string else ""
    main = soup.select_one("article") or soup.select_one("main") or soup.body
    if not main: return title, ""
    for a in main.find_all("a", href=True): a["href"] = urljoin(base_url, a["href"])
    text_md = md(str(main), strip=["img"])
    text_md = re.sub(r"\n{3,}", "\n\n", text_md).strip()
    return title, text_md

def write_md(title, url, body_md):
    os.makedirs(OUT_DIR, exist_ok=True)
    if not title: title = urlparse(url).path.strip("/").replace("/"," ") or "page"
    slug = slugify(title)[:80] or hashlib.sha1(url.encode()).hexdigest()[:10]
    path = os.path.join(OUT_DIR, f"{slug}.md")
    with open(path, "w", encoding="utf-8") as f:
        f.write("---\n")
        f.write(f'title: "{title.replace(\'"\', "\'")}"\n')
        f.write(f'url: "{url}"\nsource: "website"\n---\n\n')
        f.write(body_md.strip() + "\n")
    return path

def crawl():
    seen, queue = set(), [START_URL]
    count = 0
    while queue and count < MAX_PAGES:
        u = queue.pop(0)
        if u in seen: continue
        seen.add(u)
        if u.lower().endswith(BAD_EXT): continue
        if not same_host(u, START_URL): continue
        html = fetch(u)
        if not html: continue
        title, body = clean_html(html, u)
        if len(body) < 200: continue
        write_md(title, u, body); count += 1
        soup = BeautifulSoup(html, "html.parser")
        for a in soup.find_all("a", href=True):
            v = urljoin(u, a["href"])
            if same_host(v, START_URL) and v not in seen:
                if not v.lower().endswith(BAD_EXT):
                    queue.append(v)
        time.sleep(0.2)
    print(f"OK: {count} fichiers dans /content/{OUT_DIR}")

crawl()


SyntaxError: f-string expression part cannot include a backslash (ipython-input-943657472.py, line 48)